In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, TargetEncoder
from sklearn.metrics import roc_auc_score, confusion_matrix, classification_report
from sklearn.calibration import CalibratedClassifierCV
from tqdm import tqdm
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
import joblib
import subprocess

In [4]:
# Read the data
train = pd.read_csv('/Users/jaesolshin/내 드라이브/2024-2/Google ML Bootcamp2024/data/playground1/train.csv')
train = train.set_index('id').astype(str)

In [33]:
# submission = 다른 모델로 생성한 pseudo label
submission = pd.read_csv('/Users/jaesolshin/Documents/GitHub/kgpg_binary/submission/cat_keras_predictions.csv')
submission['Response'] = (submission['Response'] >= 0.5).astype(int)
submission = submission.set_index('id').astype(str)

# test 데이터 로드
test_origin = pd.read_csv('/Users/jaesolshin/내 드라이브/2024-2/Google ML Bootcamp2024/data/playground1/test.csv')
X_test = test_origin.set_index('id').astype(str)

In [35]:
# test = X_test + submission
test = pd.concat([X_test, submission], axis=1)

# submission = 다른 모델로 생성한 pseudo label
# (train.shape[0]*3/7)/submission.shape[0] = 0.64
submission_sampled, submission_rest = train_test_split(test, test_size=0.6, random_state=240726)
pooled = pd.concat([train, submission_sampled], axis=0)

subprocess.run(['say', '-v', 'Kyoko', 'シュドデータのサンプリング・プロセスが完了しました。'])

CompletedProcess(args=['say', '-v', 'Kyoko', 'シュドデータのサンプリング・プロセスが完了しました。'], returncode=0)

In [36]:
# 예측변수 분리
X_train = train.drop(['Response'], axis=1)
y_train = train['Response'].astype(float)
X_pooled = pooled.drop(['Response'], axis=1)
y_pooled = pooled['Response'].astype(float)

subprocess.run(['say', '-v', 'Kyoko', 'ターゲット変数の分離プロセスが完了しました。'])

CompletedProcess(args=['say', '-v', 'Kyoko', 'ターゲット変数の分離プロセスが完了しました。'], returncode=0)

In [39]:
# 타겟 인코딩
enc = TargetEncoder()
enc.fit(X_train, y_train) # 학습 데이터로 인코더 학습
subprocess.run(['say', '-v', 'Kyoko', 'インコーディングの学習プロセスが完了しました。'])

CompletedProcess(args=['say', '-v', 'Kyoko', 'インコーディングの学習プロセスが完了しました。'], returncode=0)

In [40]:
X_pooled = pd.DataFrame(enc.transform(X_pooled), index=X_pooled.index, columns=X_pooled.columns)
subprocess.run(['say', '-v', 'Kyoko', 'X_pooledのインコーディング・プロセスが完了しました。'])
print("X_pooled transform done")

X_pooled transform done


In [41]:
X_test = pd.DataFrame(enc.transform(X_test), index=X_test.index, columns=X_test.columns)
subprocess.run(['say', '-v', 'Kyoko', 'X_testのインコーディング・プロセスが完了しました。'])
print("X_test transform done")

X_test transform done


In [44]:
X_pooled = X_pooled.astype(float)
X_test = X_test.astype(float)
subprocess.run(['say', '-v', 'Kyoko', 'インコーディング・プロセスが完了しました。'])

CompletedProcess(args=['say', '-v', 'Kyoko', 'インコーディング・プロセスが完了しました。'], returncode=0)

In [ ]:
# CatBoost
print("\nCatBoost:")
cat_param = { 'loss_function': 'Logloss', 'eval_metric': 'AUC', 'class_names': [0, 1], 'learning_rate': 0.075, 'iterations': 1000, 'depth': 9, 'random_strength': 0, 'l2_leaf_reg': 0.5, 'max_leaves': 512, 'fold_permutation_block': 64, 'allow_writing_files': False, 'verbose':1}
cat_model = CatBoostClassifier(**cat_param, random_state=42)

cat_model.fit(X_pooled, y_pooled)
subprocess.run(['say', '-v', 'Kyoko', 'lgbmモデルの学習・プロセスが完了しました。'])

In [46]:
y_test_pred = lgbm_model.predict_proba(X_test)[:, 1]
submission1 = pd.DataFrame({'id': X_test.index, 'Response': y_test_pred})
subprocess.run(['say', '-v', 'Kyoko', 'lgbmモデルによる確立予測プロセスが完了しました。'])

# 예측을 CSV 파일로 저장
submission1.to_csv('/Users/jaesolshin/Documents/GitHub/kgpg_binary/submission/lgbm_pseudo_predictions.csv', index=False)
print("Predictions saved to 'lgbm_pseudo_predictions.csv'")
subprocess.run(['say', '-v', 'Kyoko', 'lgbmモデルによる提出ファイルのエクスポートプロセスが完了しました。'])

Predictions saved to 'lgbm_pseudo_predictions.csv'


CompletedProcess(args=['say', '-v', 'Kyoko', 'lgbmモデルによる提出ファイルのエクスポートプロセスが完了しました。'], returncode=0)

In [ ]:
submission2 = pd.read_csv('/Users/jaesolshin/Documents/GitHub/kgpg_binary/submission/lgbm_pseudo_predictions.csv')

In [49]:

# 'id'를 기준으로 두 DataFrame 병합
submission = pd.merge(submission1, submission2, on='id', suffixes=('_1', '_2'))

# 'Response' 열들의 평균을 계산하여 새로운 'Response' 변수 생성
submission['Response'] = submission[['Response_1', 'Response_2']].mean(axis=1)

# 원래 'Response' 변수 삭제
submission = submission.drop(columns=['Response_1', 'Response_2'])

# 예측을 CSV 파일로 저장
submission.to_csv('/Users/jaesolshin/Documents/GitHub/kgpg_binary/submission/lgb_xgb_pseudo__predictions.csv', index=False)
print("Predictions saved to 'gb_xgb__predictions.csv'")

subprocess.run(['say', '-v', 'Kyoko', 'アンサンブル・モデルによる提出ファイルのエクスポートプロセスが完了しました。'])


Predictions saved to 'gb_xgb__predictions.csv'


CompletedProcess(args=['say', '-v', 'Kyoko', 'アンサンブル・モデルによる提出ファイルのエクスポートプロセスが完了しました。'], returncode=0)